# High-Resolution Time Delay Estimation Between Noisy Signals

## Time-Domain Techniques

### Cross-Correlation and Parabolic Peak Interpolation

#### Problem statement

The cross-correlation between two signals $x(t)$ and $y(t)$ (with $y$ being a delayed version of $x$ plus noise)

which, in discrete form, becomes a sum over time shifts. The delay that maximizes $R_{xy}(\tau)$ is the estimated time difference of arrival. In practice, one computes the discrete cross-correlation (using xcorr or FFT methods) and finds the index $k_0$ of the peak. This $k_0$ gives the nearest integer-sample delay. To achieve fractional delay estimates, we can interpolate around this peak.

In [ ]:
# x, y: input signals (1D arrays), assume len(y) ≈ len(x)
import numpy as np
import matplotlib.pyplot as plt
from utils.rPPG_methods import CHROM_win

pkl_path = "data/example/highspeed_roi/sid101-rid1-qhy5iii174c-seg1-roi.pkl"
def load_xy_from_pkl(pkl_path, hs_fps=150):
    roi_df = np.load(pkl_path, allow_pickle=True)
    x = roi_df['roi-forehead'].to_list()
    y = roi_df['roi-right_cheek'].to_list()
    x = np.array(x)
    y = np.array(y)

    x = x[:,[2,1,0]] # BGR -> RGB
    y = y[:,[2,1,0]]

    x = CHROM_win(x[:,None,:], FS=hs_fps)
    y = CHROM_win(y[:,None,:], FS=hs_fps)
    x = x[0]
    y = y[0]
    return x, y
x, y = load_xy_from_pkl(pkl_path)
hs_fps = 150


#### Parabolic Peak Interpolation

A common approach is parabolic interpolation using the correlation values at the peak and its immediate neighbors. Assuming $R_{xy}(k_0)$ is the maximum at integer lag $k_0$, with neighbors $R_{xy}(k_0-1)$ and $R_{xy}(k_0+1)$, we fit a quadratic through these three points and find its vertex. If we denote $y(-1)=R_{xy}(k_0-1)$, $y(0)=R_{xy}(k_0)$, $y(1)=R_{xy}(k_0+1)$, the fractional offset $\delta$ from $k_0$ that maximizes the parabola is

In [ ]:
def frac_delay(x, y):
    # Compute cross-correlation (full mode to get negative and positive lags)
    corr = np.correlate(x, y, mode='full')  
    lags = np.arange(-len(y)+1, len(x))    # corresponding lag values

    # Find index of max correlation
    k0 = np.argmax(corr)
    peak_lag = lags[k0]        # integer lag of max correlation

    # Parabolic interpolation around the peak
    if 0 < k0 < len(corr)-1:
        y_m1, y_0, y_p1 = corr[k0-1], corr[k0], corr[k0+1]
        denom = (y_m1 - 2*y_0 + y_p1)
        if denom != 0:
            delta = (y_m1 - y_p1) / (2 * denom)
        else:
            delta = 0.0
    else:
        delta = 0.0
    frac_delay = peak_lag + delta   # estimated fractional delay (in samples)
    return frac_delay
frac_delay = frac_delay(x, y)
print(f"Fractional delay: {frac_delay*1000/hs_fps:.3f}ms")

### Sinc Interpolation and Upsampling

Rather than fitting a simple parabola, one can more precisely model the continuous cross-correlation function using band-limited interpolation. If the signals are band-limited (which is often the case if they were sampled at Nyquist rate), the cross-correlation will be a band-limited function. Sinc interpolation (reconstructing the correlation with an ideal low-pass filter) can yield an accurate fractional-delay estimate with minimal bias. In practice, ideal sinc reconstruction is not feasible, but one can **upsample** the signals or the correlation via zero-padding in frequency domain to approximate it.

- **Time-domain upsampling**: Increase the sampling rate (e.g., by polyphase filtering or inserting zeros and lowpass filtering) for one or both signals and then compute cross-correlation at the higher rate. With a high enough upsampling factor, the delay can be determined to the desired fraction of a sample. This is computationally more expensive (since signals length and correlation computations increase by the upsampling factor).
- **Frequency-domain zero-padding**: A clever trick is to compute the cross-correlation via FFT with zero-padding to achieve interpolation. For example, take FFTs of $x$ and $y$ (of length $N$), zero-pad them to length $M \gg N$, then compute the cross-correlation via inverse FFT of $X(f)Y^*(f)$. The resulting cross-correlation is effectively sampled at a finer time grid (because zero-padding in frequency corresponds to interpolation in time). This yields a dense correlation curve from which a more precise peak can be found. Note: This is equivalent to sinc interpolation of the correlation function.
- **Gaussian or Cosine interpolation**:

In [ ]:
def sinc_interp(x, y, scale_factor:int=10):
    N = len(x)
    M = scale_factor * N  # e.g., 16x upsampling in time
    X = np.fft.rfft(x, n=N)
    Y = np.fft.rfft(y, n=N)
    # Cross-spectrum and normalized inverse FFT
    cross_spec = X * np.conj(Y)

    # zero-pad them to M in frequency domain
    cross_spec = np.pad(cross_spec, (0, M//2-N//2), constant_values=(0,))
    corr = np.fft.irfft(cross_spec, n=M)  # length M correlation, contains correlation at finer steps
    # Because signals are of length N, valid correlation lags are within +/- (N-1)
    # corr array is circular; we should shift it to center = zero lag
    corr = np.roll(corr, M//2)  
    valid_lags = np.arange(-N+1, N)
    corr_valid = corr[M//2 - (N-1) : M//2 + (N-1) + 1]
    k0 = np.argmax(corr_valid)
    peak_lag_frac = valid_lags[0] + k0

    return peak_lag_frac/scale_factor

sinc_interp_factor = 10
peak_lag_frac = sinc_interp(x, y, scale_factor=sinc_interp_factor)
print(f"Sinc interpolation delay: {peak_lag_frac*1000/hs_fps:.3f}ms")


## Frequency-Domain Techniques
Frequency-domain methods exploit the fact that a time delay corresponds to a linear phase shift across frequency components. If $y(t)$ is a delayed version of $x(t)$ by $\tau$, and ignoring noise for the moment, their continuous Fourier transforms are related by $Y(f) = X(f),e^{-j 2\pi f \tau}$. This means the cross-power spectrum $S_{xy}(f) = X(f) , Y^*(f)$ contains a phase term $e^{-j2\pi f \tau}$ proportional to the delay. By analyzing the phase of the cross-spectrum (or the transfer function between $x$ and $y$), one can estimate $\tau$ with fine resolution.

### Phase Difference (Cross-Spectrum) Method (NOT WORKING WELL!)

In practice, given discrete signals, one can compute the cross-spectrum via the FFT. Let $X[k]$ and $Y[k]$ be the DFTs of $x$ and $y$. The cross-spectrum is $C[k] = X[k]\,Y^*[k]$. If $y$ is a shifted version of $x$, we expect:

$$C[k] \approx A(k)\,e^{-j\omega_k \tau},$$

where $\omega_k = 2\pi k / N$ is the digital frequency and $A(k)$ is some complex amplitude (including any gain differences). The phase of $C[k]$ is $\angle C[k] \approx -\omega_k \tau$ (mod $2\pi$). If the delay $\tau$ is small, this phase is nearly linear in $k$. We can unwrap the phase across the spectrum and perform a linear regression: the slope of $\angle C(\omega)$ vs. $\omega$ gives $-\tau$. In other words,
	•	Compute $\phi[k] = \text{unwrap}(\angle C[k])$ for $k=0…N-1$.
	•	Fit a line $\phi[k] \approx -\omega_k \tau + b$ (where $b$ is some constant phase offset). The best-fit slope in a least-squares sense yields $\hat{\tau}$.

Another equivalent view: the time delay can be estimated by the group delay between the signals, defined as $-\frac{d}{d\omega}\angle C(\omega)$. For a pure delay, this is constant and equal to the delay. In discrete form, one can compute group delay by finite difference of phase or by special algorithms.

A simple example: if $x[n]$ and $y[n]$ are sinusoids of frequency $f_0$ with a phase difference, the delay can be obtained from that phase difference divided by $2\pi f_0$. For broadband signals, the phase method uses all frequencies to improve accuracy (basically a larger aperture for estimation).

**Why it fails:**
- Phase unwrapping and ambiguity: One must unwrap the phase correctly. If the delay is large such that phase changes by more than $2\pi$ over the band, naive unwrapping might jump by $2\pi$ at wrong places. However, this is usually manageable by standard unwrapping algorithms if SNR is reasonably high.
- Requires coherent spectrum: If the two signals differ not just by delay but also by other filtering (e.g., different frequency responses or multipath effects), the phase won’t be a perfect line. That can bias the estimate unless one restricts to frequency regions where the phase is linear. In other words, this assumes the two signals are related by a pure delay (and maybe gain); any deviation (like distortion) reduces accuracy.
- Sensitive to low SNR at certain frequencies: A frequency bin with little signal (and mostly noise) will have a random phase, which can throw off the linear fit. Typically one must weight or discard frequencies with low coherence. Using a cross-spectrum coherence threshold or weighting by $|C[k]|$ can mitigate this.
- Implementation complexity is moderate: fitting a line to phase data is straightforward, but one must handle edge cases (phase wrap, outliers) carefully. It’s more complex than a simple time-domain peak search.


In [ ]:
import numpy as np

fs = hs_fps
N = len(x)
X = np.fft.rfft(x)
Y = np.fft.rfft(y)
# Assume we have cross-spectrum from earlier FFTs: cross_spec = X * np.conj(Y)
cross_spec = X * np.conj(Y)  # from previous example or computed elsewhere
freqs = np.fft.rfftfreq(N, d=1.0/fs)  # frequency axis (Fs = sampling rate)
phase = np.angle(cross_spec)
phase_unwrapped = np.unwrap(phase)

# Optionally select frequency range of interest:
# e.g., use only frequencies where |cross_spec| is significant 
mask = np.abs(cross_spec) > np.max(np.abs(cross_spec)) * 0.1  # example threshold
freqs_used = freqs[mask]
phase_used = phase_unwrapped[mask]

plt.subplot(2,1,1)
plt.plot(freqs, np.abs(cross_spec))
plt.xlim([0, 4])

plt.subplot(2,1,2)
plt.plot(freqs, phase, label='Phase')
plt.plot(freqs, phase_unwrapped, label='Unwrapped Phase')

# Plot red dotted lines for each frequency in freqs_used
for i, f_used in enumerate(freqs_used):
    if i == 0: # Add label only for the first line to avoid cluttering the legend
        plt.axvline(x=f_used, color='red', linestyle='dotted', label='Freqs Used')
    else:
        plt.axvline(x=f_used, color='red', linestyle='dotted')
plt.xlabel('Frequency (Hz)')
plt.xlim([0,4])
plt.ylim([-2*np.pi, 2*np.pi])
plt.legend()

if len(freqs_used) > 0:
    # Fit a line to phase_used vs freqs_used
    coeffs = np.polyfit(freqs_used, phase_used, 1)  # returns [slope, intercept]
    slope = coeffs[0]
    tau_est = -slope / (2*np.pi)
else:
    # If no frequencies are used, estimate tau from the first frequency and pha
    tau_est = phase_used[0]/(2*np.pi*freqs_used[0])

print(f"Frequency bins: {freqs_used}Hz, \nPhase: {phase_used}, \nEstimated time delay: {tau_est*1000:.3f}ms")

### Generalized Cross-Correlation and Phase Transform (GCC-PHAT)

The Generalized Cross-Correlation (GCC) framework is a family of techniques that modify the cross-correlation by applying a frequency-domain weighting to emphasize certain signal characteristics. The standard cross-correlation is actually one member of this family (with a flat frequency weight). By choosing a clever weight, one can improve the detection of the correlation peak in noisy or reverberant conditions.

One popular choice is the Phase Transform (PHAT) weighting. In GCC-PHAT, we whiten the spectrum by dividing by the magnitude of the cross-spectrum. The PHAT-weighted cross-correlation is computed as:

$$
R_{\text{PHAT}}(\tau) = \frac{1}{2\pi}\int_{-\pi}^{\pi} \frac{X(\omega)Y^*(\omega)}{|X(\omega)Y^*(\omega)|} \, e^{j\omega\tau}\,d\omega\,.
$$

Intuitively, we keep only the phase of each frequency’s contribution (hence “Phase Transform”) and ignore the magnitude. This has the effect of giving equal weight to all frequencies, so that no single frequency (or narrow band) dominates the correlation. If $y(t)$ is exactly $x(t-\tau_0)$ (delayed copy), and there is no other distortion, then $X(\omega)Y^*(\omega) = |X(\omega)|^2 e^{-j\omega\tau_0}$. Plugging into the PHAT formula yields:

$$
R_{\text{PHAT}}(\tau) = \frac{1}{2\pi}\int_{-\pi}^{\pi} e^{-j\omega\tau_0} e^{j\omega\tau}\,d\omega = \delta(\tau - \tau_0)\,,
$$

an ideal delta-function peak at the true delay. In discrete time, we can’t get an infinite resolution delta, but we do get a very sharp peak concentrated at the correct sample (and possibly fractional lag after interpolation).

In [ ]:
import numpy as np

# Compute FFT of signals (zero-pad to length N for efficient convolution length)
N = len(x) + len(y) - 1
X = np.fft.rfft(x, n=N)
Y = np.fft.rfft(y, n=N)
freqs = np.fft.rfftfreq(N, d=1/hs_fps)
C = X * np.conj(Y)
# Apply PHAT weighting
EPS = 1e-6
# C_phat = C   # add small EPS to avoid /0
C_phat = C / (np.abs(C) + EPS)   # add small EPS to avoid /0
gcc_phat = np.fft.irfft(C_phat, n=N)
gcc_phat = np.roll(gcc_phat, N//2)  # shift zero lag to center
# Now gcc_phat is the cross-correlation (PHAT weighted)
lag_index = np.argmax(gcc_phat)
tau_samples = lag_index - (len(x)-1)  # convert index to lag (center corresponds to 0 lag)

In [ ]:
def gcc_0(x, y, fs, tau_, PHAT_weight = False):
    # N = len(x) + len(y) - 1
    N = len(x)
    fs = hs_fps
    X = np.fft.rfft(x, n=N)
    Y = np.fft.rfft(y, n=N)
    freqs = np.fft.rfftfreq(N, d=1/fs)
    C = X * np.conj(Y)*np.exp(1j*2*np.pi*freqs*tau_)
    
    # Apply PHAT weighting
    if PHAT_weight:
        EPS = 1e-6
        C_phat = C / (np.abs(C) + EPS)   # add small EPS to avoid /0
    else:
        C_phat = C   # add small EPS to avoid /0
    gcc_phat = np.fft.irfft(C_phat, n=N)
    # gcc_phat = np.roll(gcc_phat, N//2)  # shift zero lag to center
    # Now gcc_phat is the cross-correlation (PHAT weighted)
    # return np.max(gcc_phat)
    return gcc_phat[0]

In [ ]:
tau_list = np.linspace(-.1, 0.1, 1000)
gcc_tau = list()
for tau_ in tau_list:
    gcc_tau.append(gcc_0(x, y, hs_fps, tau_, True))
print(f"Estimated tau: {tau_list[np.argmax(gcc_tau)]*1000:.3f}ms")

In [ ]:
tau_list = np.linspace(-.1, 0.1, 1000)
gcc_tau = list()
for tau_ in tau_list:
    gcc_tau.append(gcc_0(x, y, hs_fps, tau_))
print(f"Estimated tau: {tau_list[np.argmax(gcc_tau)]*1000:.3f}ms")

### Subspace Methods (MUSIC and ESPRIT)
Subspace methods like MUSIC (Multiple Signal Classification) and ESPRIT were originally developed for direction-of-arrival estimation on sensor arrays, but they can be adapted for time-delay estimation to achieve even higher resolution than classical methods​. The idea is to model the signals and noise and then use eigen-decomposition to separate the “signal subspace” (which contains the delayed copies of the waveform) from the “noise subspace.” These methods are especially powerful when multiple propagation paths (multiple delays) exist, as they can resolve distinct delays that are closer than a sample period (super-resolution). How it works (conceptually for a single delay): If $y(t)$ is a delayed version of $x(t)$, in the frequency domain we have $Y(f) = X(f)e^{-j2\pi f \tau}$. We can treat each frequency component as a sensor observing the phase-shifted signal. For example, take a set of $M$ frequency bins ${f_1, f_2, ..., f_M}$ where the signal has significant energy. The phase of $Y$ relative to $X$ at those bins is $-2\pi f_k \tau$. This forms a model akin to an array of $M$ sensors (frequencies) with a single source at “angle” (delay) $\tau$. MUSIC can be applied by constructing a covariance matrix of the cross-spectral data across these $M$ “sensors.” The covariance will have a rank-1 signal subspace (for one delay) plus noise. MUSIC then scans a candidate $\tau$ and evaluates a pseudo-spectrum $P(\tau) = \frac{1}{\mathbf{a}^H(\tau) \mathbf{N} \mathbf{N}^H \mathbf{a}(\tau)}$ where $\mathbf{N}$ is the noise subspace eigenvectors and $\mathbf{a}(\tau)$ is the array steering vector $[1, e^{-j2\pi f_2 \tau}, ..., e^{-j2\pi f_M \tau}]^T$. Peaks of $P(\tau)$ indicate likely delays (where the candidate steering vector aligns with the signal subspace)​. ESPRIT, on the other hand, can find the delays algebraically by exploiting invariances (often requiring paired sensors or frequencies). In the time domain, one can also create multiple “sensors” by taking delayed versions of the signal (e.g., constructing a covariance matrix from $[x(t), y(t)]$ as two sensors, or using lagged versions as additional sensors via covariance lags or sub-band averaging​). In ultrawideband scenarios or radar with multiple echoes, one can often partition the spectrum or time into sub-bands which act as multiple measurements for the subspace algorithm​. These details can get mathematically heavy, but the end result is an estimate of one or more delays with very high precision.

In [ ]:
N = len(x)
X = np.fft.rfft(x,N)
Y = np.fft.rfft(y,N)
freqs = np.fft.rfftfreq(N, 1/hs_fps)

In [ ]:
Z = X*Y.conj()
R = np.outer(Z, Z.conj())
eigvals, eigvecs = np.linalg.eigh(R)

In [ ]:
En = eigvecs[:, :-1]
delay_grid = np.linspace(-.1, .1, 500)
# delay_grid = np.linspace(-0.1, 0.1, 100)
pseudospectrum = []
for tau_ in delay_grid:
    a = np.exp(-1j * 2 * np.pi * tau_ * freqs)
    a = a[:, np.newaxis]
    denom = np.abs(a.conj().T @ En @ En.conj().T @ a)[0,0]
    pseudospectrum.append(1/denom)


In [ ]:
print(delay_grid[np.argmax(pseudospectrum)])
plt.plot(delay_grid, np.array(pseudospectrum))

In [ ]:
a = np.exp(-1j * 2 * np.pi * tau_ * freqs)
a_abs = a*np.abs(Z)

In [ ]:
En = eigvecs[:, -1:]
# delay_grid = np.linspace(-1, 1, 100)
delay_grid = np.linspace(-0.1, 0.1, 100)
pseudospectrum = []
for tau_ in delay_grid:
    a = np.exp(-1j * 2 * np.pi * tau_ * freqs)
    a = a[:, np.newaxis]
    denom = np.abs(a.conj().T @ En @ En.conj().T @ a)[0,0]
    pseudospectrum.append(denom)

print(delay_grid[np.argmax(pseudospectrum)])
plt.plot(delay_grid, np.array(pseudospectrum))

pseudospectrum = []
for tau_ in delay_grid:
    a = np.exp(-1j * 2 * np.pi * tau_ * freqs)*np.abs(Z)
    a = a[:, np.newaxis]
    denom = np.abs(a.conj().T @ En @ En.conj().T @ a)[0,0]
    pseudospectrum.append(denom)

print(delay_grid[np.argmax(pseudospectrum)])
plt.plot(delay_grid, np.array(pseudospectrum))

#### Sub-band MUSIC

In [ ]:
# MUSIC applied on selected frequency bins
def music_on_selected_band(x, y, fs, freq_band, delay_grid):
    N = len(x)
    X = np.fft.rfft(x)
    Y = np.fft.rfft(y)
    freqs = np.fft.rfftfreq(N, d=1/fs)
    # Select frequency bins
    bin_mask = (freqs >= freq_band[0]) & (freqs <= freq_band[1])
    X_sel = X[bin_mask]
    Y_sel = Y[bin_mask]
    freqs_sel = freqs[bin_mask]
    # Cross-spectrum
    Z = X_sel * np.conj(Y_sel)
    R = np.outer(Z, Z.conj())
    # Eigen-decomposition
    eigvals, eigvecs = np.linalg.eigh(R)
    En = eigvecs[:, :-1]  # assume 1 delay
    # MUSIC pseudospectrum
    pseudospectrum = []
    for tau in delay_grid:
        a = np.exp(-1j * 2 * np.pi * freqs_sel * tau)[:, np.newaxis]
        denom = np.abs(a.conj().T @ En @ En.conj().T @ a)[0, 0]
        pseudospectrum.append(1 / denom)
    return np.real(pseudospectrum)

In [ ]:
delay_grid = np.linspace(-.1, .1, 500)
freq_band = [0.7, 2.0]
# freq_band = [1.4, 1.5]
fs = hs_fps
pseudo_spectrum_narrow_band = music_on_selected_band(x, y, fs, freq_band, delay_grid)
print(delay_grid[np.argmax(pseudo_spectrum_narrow_band)])
plt.plot(delay_grid, pseudo_spectrum_narrow_band)

# Benchmark on Thai HS dataset

## Time Delay Estimation

In [ ]:
import importlib

# Import the module
time_delay_estimation = importlib.import_module('time_delay_estimation')

# Reload the module to ensure any changes are applied
time_delay_estimation = importlib.reload(time_delay_estimation)

# Define the list of function names
delay_estimation_methods = ['frac_delay', 'sinc_interp', 'gcc_delay']

# Create a dictionary of functions using dictionary comprehension
delay_estimation_functions = {
    method: getattr(time_delay_estimation, method) 
    for method in delay_estimation_methods
}

# Setting parameters for the delay estimation
tau_grid = np.linspace(-.1,.1, 1000)
scale_factor = 16
PHAT_weight = False

fs = 150

In [ ]:
def load_xy_from_pkl(pkl_path, hs_fps=150):
    roi_df = np.load(pkl_path, allow_pickle=True)
    x = roi_df['roi-forehead'].to_list()
    y = roi_df['roi-right_cheek'].to_list()
    ts = roi_df['timestamp'].to_list()

    x = np.array(x)
    y = np.array(y)
    ts = np.array(ts)
    ts-=ts[0]

    x = x[:,[2,1,0]] # BGR -> RGB
    y = y[:,[2,1,0]]

    x = CHROM_win(x[:,None,:], FS=hs_fps)
    y = CHROM_win(y[:,None,:], FS=hs_fps)
    x = x[0]
    y = y[0]
    return (x, y), ts

def load_roi_from_pkl(pkl_path, hs_fps=150):
    roi_df = np.load(pkl_path, allow_pickle=True)
    roi_list = ['forehead', 'glabella', 'left_cheek', 'right_cheek', 'chin']
    roi_dict = {}
    for roi_ in roi_list:
        roi_arr = np.array(roi_df['roi-forehead'].to_list())
        roi_arr = roi_arr[:,[2,1,0]] # BGR -> RGB
        roi_arr = CHROM_win(roi_arr[:,None,:], FS=hs_fps)[0]
        roi_dict[roi_] = roi_arr
    return roi_dict

In [ ]:
import re
import os
import pandas as pd
from glob import glob
from collections import defaultdict
from module_ptt_estimator.extractor import PTTExtractor

roi_path = r"C:\Users\Shootall\Documents\rPPG-dataset\highspeed_roi"
bp_path = r"C:\Users\Shootall\Documents\rPPG-dataset\highspeed_bp\highspeed_bp"
roi_file_list = glob(os.path.join(roi_path, "*.pkl"))
bp_file_list = glob(os.path.join(bp_path, "*.pkl"))

pttExt = PTTExtractor()

delay_df = defaultdict(list)

for f_ in roi_file_list:
    (x, y), ts = load_xy_from_pkl(f_, hs_fps=fs)
    match_name = re.search('sid(\d+)-rid(\d+)-qhy5iii174c-seg(\d+)-roi.pkl', f_).groups()
    sid, rid, i_seg = match_name
    print(f"Working on {match_name}")

    delay_df['sid'].append(sid)
    delay_df['rid'].append(rid)
    delay_df['i_seg'].append(i_seg)

    bp_df = np.load(os.path.join(bp_path, f"sid{sid}-rid{rid}-uscom_bp_plus-seg{i_seg}-reference.pkl"), allow_pickle=True)
    if bp_df.shape[0] > 0:
        delay_df['sbp'].append(bp_df.iloc[0]['Log_Sys'])
        delay_df['dbp'].append(bp_df.iloc[0]['Log_Dia'])
    else:
        delay_df['sbp'].append(np.nan)
        delay_df['dbp'].append(np.nan)

    for de_method in delay_estimation_methods:
        delay = delay_estimation_functions[de_method](x, y, fs, scale_factor=scale_factor, tau_grid=tau_grid, PHAT_weight=PHAT_weight)
        # print(f"{de_method}: {delay:.4f} seconds")
        delay_df[de_method].append(delay)
    pai_ptt_peak = pttExt.ptt_via_peak2peak(ts, x, ts, y,)
    delay_df['pai_ptt_peak'].append(np.mean(pai_ptt_peak['valid_ptt_list']))
    pai_ptt_slope = pttExt.ptt_via_slope2slope(ts, x, ts, y)
    delay_df['pai_ptt_slope'].append(np.mean(pai_ptt_slope['valid_ptt_list']))
    pai_ptt_corr = pttExt.ptt_via_slope2slope(ts, x, ts, y)
    delay_df['pai_ptt_corr'].append(np.mean(pai_ptt_corr['valid_ptt_list']))

delay_df = pd.DataFrame(delay_df)

In [91]:
delay_df

,sid,rid,i_seg,sbp,dbp,frac_delay,sinc_interp,gcc_delay,pai_ptt_peak,pai_ptt_slope,pai_ptt_corr
0,100,1,1,148.0,93.0,0.013950,0.013750,0.013914,0.055883,0.073396,0.073396
1,100,1,2,148.0,93.0,0.009128,0.009167,0.009109,0.059281,0.034645,0.034645
2,100,1,3,148.0,93.0,0.013224,0.013333,0.013313,0.056898,0.041237,0.041237
3,100,1,4,148.0,93.0,0.017688,0.017500,0.017718,0.051828,0.023661,0.023661
4,100,1,5,148.0,93.0,0.021274,0.021250,0.021321,0.057837,0.037856,0.037856
...,...,...,...,...,...,...,...,...,...,...,...
1362,99,3,1,126.0,87.0,0.001326,0.001250,0.001301,0.066696,0.040558,0.040558
1363,99,3,2,126.0,87.0,-0.000946,-0.000833,-0.000901,0.079186,0.063252,0.063252
1364,99,3,3,126.0,87.0,0.008013,0.007917,0.008108,0.081568,0.067598,0.067598
1365,99,3,4,126.0,87.0,-0.013053,-0.012917,-0.013113,0.063910,0.049918,0.049918


In [93]:
1/np.mean(np.diff(ts))

147.93435822650756